# Notebook 1 - Carga de Datos

## Objetivo y alcance
- Que hace este notebook: cargar, validar tecnicamente y exportar datasets base.
- Resultado esperado: un dataset canonico confiable para `Notebook_2` y artefactos de control reproducibles.


## Restricciones tecnicas de librerias permitidas
Librerias permitidas por el enunciado: `numpy`, `pandas`, `yfinance`, `matplotlib`, `seaborn`, `scipy`, `pyarrow`.
Ademas se usa libreria estandar de Python: `pathlib`, `json`, `datetime`, `sys`, `platform`.


In [1]:
# Importa calculo numerico vectorizado (libreria permitida).
import numpy as np
# Importa estructuras tabulares y utilidades de datos (libreria permitida).
import pandas as pd
# Importa motor parquet permitido para lectura/escritura.
import pyarrow as pa
# Importa proveedor de datos de mercado permitido.
import yfinance as yf
# Importa manejo de rutas de forma portable.
from pathlib import Path
# Importa serializacion JSON para artefactos de config y reportes.
import json
# Importa fecha/hora con zona UTC para trazabilidad reproducible.
from datetime import datetime, timezone
# Importa informacion de version del interprete Python.
import sys
# Importa informacion de plataforma para trazabilidad del entorno.
import platform

# Define helper fail-fast para cortar ejecucion cuando una condicion invalida aparece.
def fail_if(condition: bool, message: str) -> None:
    """Lanza ValueError si la condicion recibida es verdadera."""
    # Evalua la condicion de fallo.
    if condition:
        # Lanza error explicito para detener pipeline y evitar resultados corruptos.
        raise ValueError(message)


# Define helper para registrar checks y aplicar fail-fast en una sola llamada.
def record_check(results: list, check_name: str, failed: bool, details: str) -> None:
    """Registra el estado de un check y detiene si falla."""
    # Calcula estado booleano de aprobacion.
    passed = not failed
    # Agrega una fila al reporte de validacion.
    results.append({"check": check_name, "passed": passed, "details": details})
    # Si el check fallo, corta ejecucion inmediatamente.
    fail_if(failed, f"[{check_name}] {details}")


In [2]:
# Define parametros maestros del proyecto como fuente unica.
PARAMS = {
    # Define capital inicial del enunciado.
    "initial_capital": 250000,
    # Define inicio oficial del backtest.
    "backtest_start": "2015-01-01",
    # Define ticker benchmark oficial.
    "benchmark_ticker": "SPY",
    # Define regla de universo point-in-time para evitar look-ahead.
    "universe_rule": "in_sp500_point_in_time_last_13_months",
    # Define coste proporcional de transaccion para etapas posteriores.
    "transaction_fee_rate": 0.0023,
    # Define coste minimo por orden para etapas posteriores.
    "min_fee_per_order": 23.0,
}

# Muestra parametros para auditoria humana rapida.
PARAMS


{'initial_capital': 250000,
 'backtest_start': '2015-01-01',
 'benchmark_ticker': 'SPY',
 'universe_rule': 'in_sp500_point_in_time_last_13_months',
 'transaction_fee_rate': 0.0023,
 'min_fee_per_order': 23.0}

In [3]:
# Define raiz del proyecto como directorio actual resuelto.
PROJECT_ROOT = Path.cwd().resolve()
# Define posibles ubicaciones del parquet de entrada con prioridad local.
INPUT_CANDIDATES = [
    # Prioriza una copia local dentro del proyecto para maxima portabilidad.
    PROJECT_ROOT / "data" / "raw" / "sp500_history.parquet",
    # Usa ruta de descarga del usuario como fallback de conveniencia.
    Path.home() / "Downloads" / "sp500_history.parquet",
]
# Selecciona la primera ruta existente entre candidatas.
INPUT_PARQUET = next((path for path in INPUT_CANDIDATES if path.exists()), None)
# Detiene pipeline si no se encontro ningun input valido.
fail_if(INPUT_PARQUET is None, f"Input parquet no encontrado en: {[str(path) for path in INPUT_CANDIDATES]}")

# Define helper para guardar rutas relativas al proyecto cuando sea posible.
def to_rel(path: Path) -> str:
    """Devuelve ruta relativa al proyecto; si no se puede, devuelve absoluta."""
    # Intenta convertir la ruta a relativa respecto al root del proyecto.
    try:
        # Retorna ruta relativa para mejorar portabilidad entre entornos.
        return str(path.resolve().relative_to(PROJECT_ROOT))
    # Si falla la conversion, usa ruta absoluta como fallback.
    except ValueError:
        # Retorna ruta absoluta para no perder trazabilidad.
        return str(path.resolve())


# Define carpeta de datos crudos para artefactos sin transformar en exceso.
DIR_RAW = PROJECT_ROOT / "data" / "raw"
# Define carpeta de datos procesados para consumo del siguiente notebook.
DIR_PROCESSED = PROJECT_ROOT / "data" / "processed"
# Define carpeta de configuracion comun entre notebooks.
DIR_CONFIG = PROJECT_ROOT / "config"
# Define carpeta de reportes tecnicos de validacion.
DIR_REPORTS = PROJECT_ROOT / "reports"
# Crea carpetas si no existen para asegurar estructura estable del pipeline.
for directory in [DIR_RAW, DIR_PROCESSED, DIR_CONFIG, DIR_REPORTS]:
    # Crea directorio y padres necesarios sin error si ya existe.
    directory.mkdir(parents=True, exist_ok=True)

# Define salida canonical para N2.
OUT_CANONICAL = DIR_PROCESSED / "sp500_canonical.parquet"
# Define salida benchmark SPY descargado.
OUT_SPY = DIR_RAW / "spy_yfinance.parquet"
# Define salida de config comun para N1-N5.
OUT_CONFIG = DIR_CONFIG / "config.json"
# Define salida de reporte tecnico JSON.
OUT_REPORT_JSON = DIR_REPORTS / "data_quality_n1.json"
# Define salida de reporte tecnico CSV.
OUT_REPORT_CSV = DIR_REPORTS / "data_quality_n1_checks.csv"
# Define salida de resumen de cobertura en CSV.
OUT_COVERAGE_CSV = DIR_REPORTS / "data_quality_n1_coverage.csv"

# Registra timestamp UTC de ejecucion para reproducibilidad.
RUN_TS_UTC = datetime.now(timezone.utc).isoformat()
# Registra versiones tecnicas clave del entorno.
VERSIONS = {
    # Registra version de Python.
    "python": sys.version.split()[0],
    # Registra version de plataforma.
    "platform": platform.platform(),
    # Registra version de numpy.
    "numpy": np.__version__,
    # Registra version de pandas.
    "pandas": pd.__version__,
    # Registra version de pyarrow.
    "pyarrow": pa.__version__,
    # Registra version de yfinance.
    "yfinance": yf.__version__,
}

# Construye bloque de trazabilidad minima requerido.
TRACEABILITY = {
    # Guarda timestamp UTC de ejecucion.
    "run_ts_utc": RUN_TS_UTC,
    # Guarda input seleccionado para ejecucion.
    "input_parquet": to_rel(INPUT_PARQUET),
    # Guarda listado de rutas candidatas de input.
    "input_candidates": [to_rel(path) for path in INPUT_CANDIDATES],
    # Guarda ruta de salida canonical.
    "out_canonical": to_rel(OUT_CANONICAL),
    # Guarda ruta de salida benchmark.
    "out_spy": to_rel(OUT_SPY),
    # Guarda versiones de librerias y entorno.
    "versions": VERSIONS,
}

# Muestra trazabilidad para auditoria.
TRACEABILITY



{'run_ts_utc': '2026-02-17T18:18:28.891764+00:00',
 'input_parquet': 'data\\raw\\sp500_history.parquet',
 'input_candidates': ['data\\raw\\sp500_history.parquet',
  'C:\\Users\\dgall\\Downloads\\sp500_history.parquet'],
 'out_canonical': 'data\\processed\\sp500_canonical.parquet',
 'out_spy': 'data\\raw\\spy_yfinance.parquet',
 'versions': {'python': '3.13.5',
  'platform': 'Windows-11-10.0.26200-SP0',
  'numpy': '2.1.3',
  'pandas': '2.2.3',
  'pyarrow': '19.0.0',
  'yfinance': '1.1.0'}}

In [4]:
# Define columnas esperadas para carga robusta y consistente.
EXPECTED_COLUMNS = [
    "date",
    "symbol",
    "assetid",
    "security_name",
    "sector",
    "industry",
    "subsector",
    "in_sp500",
    "open",
    "high",
    "low",
    "close",
    "volume",
    "unadjusted_close",
]

# Verifica que el parquet de entrada exista antes de leer.
fail_if(not INPUT_PARQUET.exists(), f"Input parquet no encontrado: {INPUT_PARQUET}")
# Lee parquet con seleccion explicita de columnas esperadas.
df = pd.read_parquet(INPUT_PARQUET, columns=EXPECTED_COLUMNS)
# Normaliza nombres de columnas a minusculas para homogeneidad.
df.columns = [str(col).lower() for col in df.columns]
# Convierte columna date a datetime y marca errores como NaT para validacion posterior.
df["date"] = pd.to_datetime(df["date"], errors="coerce")
# Fuerza symbol a dtype string de pandas para estabilidad en merges y filtros.
df["symbol"] = df["symbol"].astype("string")
# Ordena por fecha y simbolo para garantizar consistencia temporal.
df = df.sort_values(["date", "symbol"]).reset_index(drop=True)

# Muestra forma del dataset base cargado.
df.shape


(7250110, 14)

In [5]:
# Inicializa contenedor de checks tecnicos.
validation_rows = []

# Verifica presencia de todas las columnas requeridas.
missing_columns = sorted(set(EXPECTED_COLUMNS) - set(df.columns))
# Registra check de columnas obligatorias.
record_check(
    validation_rows,
    "required_columns_present",
    bool(missing_columns),
    f"missing_columns={missing_columns}",
)

# Verifica tipo datetime en date.
date_is_datetime = pd.api.types.is_datetime64_any_dtype(df["date"])
# Registra check de tipo para date.
record_check(validation_rows, "date_is_datetime", not date_is_datetime, "date dtype invalido")

# Verifica tipo string para symbol.
symbol_is_string = pd.api.types.is_string_dtype(df["symbol"])
# Registra check de tipo para symbol.
record_check(validation_rows, "symbol_is_string", not symbol_is_string, "symbol dtype invalido")

# Define columnas que deben ser numericas.
numeric_columns = ["in_sp500", "open", "high", "low", "close", "volume", "unadjusted_close"]
# Detecta columnas numericas con tipo incorrecto.
numeric_type_failures = [col for col in numeric_columns if not pd.api.types.is_numeric_dtype(df[col])]
# Registra check de tipos numericos.
record_check(
    validation_rows,
    "numeric_columns_dtype",
    bool(numeric_type_failures),
    f"invalid_numeric_dtype={numeric_type_failures}",
)

# Calcula duplicados de clave primaria tecnica date+symbol.
duplicate_count = int(df.duplicated(["date", "symbol"]).sum())
# Registra check de duplicados clave.
record_check(
    validation_rows,
    "duplicate_date_symbol",
    duplicate_count > 0,
    f"duplicate_rows={duplicate_count}",
)

# Define columnas criticas para continuidad del pipeline.
critical_columns = ["date", "symbol", "in_sp500", "open", "high", "low", "close", "volume"]
# Calcula nulos sobre columnas criticas.
critical_null_count = int(df[critical_columns].isna().sum().sum())
# Registra check de nulos criticos.
record_check(
    validation_rows,
    "critical_nulls",
    critical_null_count > 0,
    f"critical_null_count={critical_null_count}",
)

# Calcula filas con precios OHLC no positivos.
invalid_price_count = int((df[["open", "high", "low", "close"]] <= 0).any(axis=1).sum())
# Registra check de precios positivos.
record_check(
    validation_rows,
    "prices_positive",
    invalid_price_count > 0,
    f"invalid_price_rows={invalid_price_count}",
)

# Calcula filas con volumen negativo.
invalid_volume_count = int((df["volume"] < 0).sum())
# Registra check de volumen no negativo.
record_check(
    validation_rows,
    "volume_non_negative",
    invalid_volume_count > 0,
    f"invalid_volume_rows={invalid_volume_count}",
)

# Calcula condicion de OHLC incoherente por limite superior.
high_inconsistency = df["high"] < df[["open", "close", "low"]].max(axis=1)
# Calcula condicion de OHLC incoherente por limite inferior.
low_inconsistency = df["low"] > df[["open", "close", "high"]].min(axis=1)
# Calcula total de inconsistencias OHLC.
invalid_ohlc_count = int((high_inconsistency | low_inconsistency).sum())
# Registra check de coherencia OHLC.
record_check(
    validation_rows,
    "ohlc_consistency",
    invalid_ohlc_count > 0,
    f"invalid_ohlc_rows={invalid_ohlc_count}",
)

# Calcula filas con valores de in_sp500 fuera de {0, 1}.
invalid_in_sp500_count = int((~df["in_sp500"].isin([0, 1])).sum())
# Registra check de dominio binario para in_sp500.
record_check(
    validation_rows,
    "in_sp500_binary",
    invalid_in_sp500_count > 0,
    f"invalid_in_sp500_rows={invalid_in_sp500_count}",
)

# Convierte resultados de checks a dataframe para export y lectura humana.
checks_df = pd.DataFrame(validation_rows)
# Muestra resumen de validaciones tecnicas.
checks_df


,check,passed,details
0,required_columns_present,True,missing_columns=[]
1,date_is_datetime,True,date dtype invalido
2,symbol_is_string,True,symbol dtype invalido
3,numeric_columns_dtype,True,invalid_numeric_dtype=[]
4,duplicate_date_symbol,True,duplicate_rows=0
5,critical_nulls,True,critical_null_count=0
6,prices_positive,True,invalid_price_rows=0
7,volume_non_negative,True,invalid_volume_rows=0
8,ohlc_consistency,True,invalid_ohlc_rows=0
9,in_sp500_binary,True,invalid_in_sp500_rows=0


In [6]:
# Calcula fecha minima del dataset cargado.
date_min = df["date"].min()
# Calcula fecha maxima del dataset cargado.
date_max = df["date"].max()
# Calcula numero total de simbolos historicos.
total_symbols = int(df["symbol"].nunique())
# Calcula ultimo dia disponible del dataset.
last_data_date = date_max
# Calcula simbolos con datos en la fecha maxima.
symbols_on_last_date = int(df.loc[df["date"] == last_data_date, "symbol"].nunique())
# Calcula ultima fecha observada por simbolo para estimar actividad.
last_date_by_symbol = df.groupby("symbol", observed=True)["date"].max()
# Calcula simbolos activos en la ultima fecha disponible.
active_symbols = int((last_date_by_symbol == last_data_date).sum())
# Calcula simbolos historicos que ya no llegan a la ultima fecha.
inactive_symbols = int(total_symbols - active_symbols)
# Calcula serie diaria del conteo de miembros S&P por flag point-in-time.
sp500_daily_count = (
    df.loc[df["in_sp500"] == 1]
    .groupby("date", observed=True)["symbol"]
    .nunique()
    .rename("sp500_members")
)

# Construye resumen compacto de cobertura basica.
coverage_summary = {
    "date_min": date_min.date().isoformat(),
    "date_max": date_max.date().isoformat(),
    "total_rows": int(len(df)),
    "total_symbols": total_symbols,
    "symbols_on_last_date": symbols_on_last_date,
    "active_symbols": active_symbols,
    "inactive_symbols": inactive_symbols,
    "sp500_members_min": int(sp500_daily_count.min()),
    "sp500_members_median": float(sp500_daily_count.median()),
    "sp500_members_max": int(sp500_daily_count.max()),
}

# Muestra resumen de cobertura para control rapido.
coverage_summary


{'date_min': '1990-01-02',
 'date_max': '2026-01-30',
 'total_rows': 7250110,
 'total_symbols': 1289,
 'symbols_on_last_date': 650,
 'active_symbols': 650,
 'inactive_symbols': 639,
 'sp500_members_min': 498,
 'sp500_members_median': 500.0,
 'sp500_members_max': 507}

## Nota metodologica: point-in-time y sesgo de supervivencia
- El universo se construira por fecha usando `in_sp500` en cada momento historico (point-in-time).
- Se conservan simbolos que dejaron de cotizar o salieron del indice para no introducir sesgo de supervivencia.
- El objetivo es evitar usar informacion futura que no estaria disponible en cada fecha de decision.


## Convencion de precios para etapas posteriores
- Señales de momentum: se usara `close` (ajustado) para retornos comparables historicamente.
- Ejecucion de rebalanceo: se usaran `open` y `close` segun protocolo del enunciado.
- Auditoria: se conserva `unadjusted_close` para trazabilidad y comprobaciones adicionales.


In [7]:
# Inicializa fuente de benchmark para trazabilidad operativa.
spy_source = "yfinance_download"
# Intenta descargar benchmark SPY desde yfinance.
try:
    # Descarga OHLCV desde inicio de backtest con ajuste desactivado.
    spy_raw = yf.download(
        tickers=PARAMS["benchmark_ticker"],
        start=PARAMS["backtest_start"],
        progress=False,
        auto_adjust=False,
    )
    # Verifica que la descarga devolvio datos.
    fail_if(spy_raw.empty, "No se pudo descargar SPY desde yfinance.")
    # Si yfinance devuelve columnas multinivel, aplana al primer nivel.
    if isinstance(spy_raw.columns, pd.MultiIndex):
        # Conserva solo nombre de campo OHLCV del primer nivel.
        spy_raw.columns = [str(col[0]).lower().replace(" ", "_") for col in spy_raw.columns]
    else:
        # Normaliza columnas simples a minusculas y snake_case.
        spy_raw.columns = [str(col).lower().replace(" ", "_") for col in spy_raw.columns]
    # Mueve el indice Date a columna para homogeneizar con el parquet principal.
    spy_df = spy_raw.reset_index()
    # Normaliza nombres de columnas tras reset del indice.
    spy_df.columns = [str(col).lower().replace(" ", "_") for col in spy_df.columns]
    # Fuerza parseo de fecha para control de calidad basico.
    spy_df["date"] = pd.to_datetime(spy_df["date"], errors="coerce")
    # Inserta identificador de simbolo benchmark.
    spy_df["symbol"] = PARAMS["benchmark_ticker"]
# Si falla la descarga, usa fallback local para permitir re-ejecucion sin red.
except Exception as exc:
    # Marca fuente fallback para trazabilidad.
    spy_source = "local_parquet_fallback"
    # Detiene pipeline si tampoco existe benchmark local previo.
    fail_if(not OUT_SPY.exists(), f"No se pudo descargar SPY y no existe fallback local: {exc}")
    # Carga benchmark local previamente guardado.
    spy_df = pd.read_parquet(OUT_SPY)

# Define columnas minimas requeridas para benchmark base.
spy_required_columns = ["date", "symbol", "open", "high", "low", "close", "volume"]
# Detecta columnas faltantes en benchmark.
spy_missing_columns = sorted(set(spy_required_columns) - set(spy_df.columns))
# Registra check tecnico del benchmark disponible.
record_check(
    validation_rows,
    "spy_required_columns_present",
    bool(spy_missing_columns),
    f"spy_missing_columns={spy_missing_columns}",
)
# Fuerza parseo de fecha en benchmark por consistencia.
spy_df["date"] = pd.to_datetime(spy_df["date"], errors="coerce")
# Selecciona columnas canonical del benchmark y ordena por fecha.
spy_df = spy_df[spy_required_columns].sort_values("date").reset_index(drop=True)
# Guarda benchmark en parquet para notebooks posteriores.
spy_df.to_parquet(OUT_SPY, index=False)
# Registra fuente final de benchmark en trazabilidad.
TRACEABILITY["spy_source"] = spy_source

# Muestra muestra corta del benchmark cargado.
spy_df.head()



,date,symbol,open,high,low,close,volume
0,2015-01-02,SPY,206.380005,206.880005,204.179993,205.429993,121465900
1,2015-01-05,SPY,204.169998,204.369995,201.350006,201.720001,169632600
2,2015-01-06,SPY,202.089996,202.720001,198.860001,199.820007,209151400
3,2015-01-07,SPY,201.419998,202.720001,200.880005,202.309998,125346700
4,2015-01-08,SPY,204.009995,206.160004,203.990005,205.899994,147217800


In [8]:
# Guarda dataset canonico validado para Notebook 2.
df.to_parquet(OUT_CANONICAL, index=False)

# Actualiza parametros con fecha final real disponible en datos.
params_with_data_end = {
    **PARAMS,
    "backtest_end_data": date_max.date().isoformat(),
}

# Construye payload de configuracion comun para N1-N5.
config_payload = {
    "project": "diseno_algoritmos_y_backtesting_avanzado",
    "params": params_with_data_end,
    "paths": {
        "input_parquet": to_rel(INPUT_PARQUET),
        "canonical_parquet": to_rel(OUT_CANONICAL),
        "benchmark_parquet": to_rel(OUT_SPY),
        "quality_json": to_rel(OUT_REPORT_JSON),
        "quality_csv": to_rel(OUT_REPORT_CSV),
        "coverage_csv": to_rel(OUT_COVERAGE_CSV),
    },
    "price_conventions": {
        "signals_price": "close",
        "execution_prices": ["open", "close"],
        "audit_price": "unadjusted_close",
    },
    "traceability": TRACEABILITY,
}
# Escribe config comun en formato JSON legible.
OUT_CONFIG.write_text(json.dumps(config_payload, indent=2), encoding="utf-8")

# Reconstruye dataframe de checks por si se agregaron checks posteriores (ej: SPY).
checks_df = pd.DataFrame(validation_rows)
# Construye payload de reporte tecnico de calidad.
quality_report_payload = {
    "run_ts_utc": RUN_TS_UTC,
    "traceability": TRACEABILITY,
    "coverage_summary": coverage_summary,
    "checks": checks_df.to_dict(orient="records"),
}
# Escribe reporte JSON de validacion tecnica.
OUT_REPORT_JSON.write_text(json.dumps(quality_report_payload, indent=2), encoding="utf-8")
# Escribe tabla CSV de checks tecnicos.
checks_df.to_csv(OUT_REPORT_CSV, index=False)
# Escribe tabla CSV de cobertura basica.
pd.DataFrame([coverage_summary]).to_csv(OUT_COVERAGE_CSV, index=False)

# Muestra resumen de artefactos exportados.
artifacts_summary = {
    "canonical_exists": OUT_CANONICAL.exists(),
    "spy_exists": OUT_SPY.exists(),
    "config_exists": OUT_CONFIG.exists(),
    "quality_json_exists": OUT_REPORT_JSON.exists(),
    "quality_csv_exists": OUT_REPORT_CSV.exists(),
    "coverage_csv_exists": OUT_COVERAGE_CSV.exists(),
}
# Muestra estado de exportes.
artifacts_summary



{'canonical_exists': True,
 'spy_exists': True,
 'config_exists': True,
 'quality_json_exists': True,
 'quality_csv_exists': True,
 'coverage_csv_exists': True}

In [9]:
# Construye checklist final de readiness para permitir pasar a Notebook 2.
readiness_checklist = {
    "all_critical_checks_passed": bool(checks_df["passed"].all()),
    "canonical_dataset_created": OUT_CANONICAL.exists(),
    "benchmark_dataset_created": OUT_SPY.exists(),
    "config_json_created": OUT_CONFIG.exists(),
    "quality_json_created": OUT_REPORT_JSON.exists(),
    "quality_csv_created": OUT_REPORT_CSV.exists(),
}
# Convierte checklist a dataframe para lectura clara en notebook.
readiness_df = pd.DataFrame(
    [{"item": key, "passed": value} for key, value in readiness_checklist.items()]
)
# Muestra checklist final.
readiness_df

# Corta ejecucion si el notebook no queda apto para continuar pipeline.
fail_if(not all(readiness_checklist.values()), "Data readiness FAILED: revisar checks y artefactos.")
# Imprime mensaje final de aprobacion formal.
print("DATA READINESS: PASS. El proyecto puede continuar en Notebook_2.")


DATA READINESS: PASS. El proyecto puede continuar en Notebook_2.
